In [1]:
import pandas as pd
import pickle
from src.prompt import *
import pickle

In [2]:
with open("/home/yl3427/cylab/SOAP_MA/ltm_build_memory_at_end.pkl", "rb") as f:
    data = pickle.load(f)


for general_ltm in data[0][:5]:
    print(general_ltm)
    print("=====================================")
print("=====================================")

for specific_ltm in data[1][:5]:
    print(specific_ltm)
    print("=====================================")

Consider the patient's entire medical history, including past diagnoses, treatments, and hospitalizations, when assessing their current condition.
Maintain a broad differential diagnosis and avoid premature closure, considering multiple potential causes for a patient's symptoms.
Be aware of potential biases and gaps in information, and actively seek out additional data to inform diagnostic decisions.
Integrate information from multiple sources, including laboratory results, imaging studies, and clinical observations, to form a comprehensive understanding of the patient's condition.
Prioritize the identification and management of life-threatening and critical conditions over potential but less certain diagnoses.
Be aware of potential complications and comorbidities in patients with end-stage renal disease (ESRD) and end-stage liver disease (ESLD), such as pulmonary hypertension and acute respiratory distress syndrome (ARDS).
Consider infectious etiologies, such as influenza, in patients

In [4]:
data[1]

['Be aware of potential complications and comorbidities in patients with end-stage renal disease (ESRD) and end-stage liver disease (ESLD), such as pulmonary hypertension and acute respiratory distress syndrome (ARDS).',
 'Consider infectious etiologies, such as influenza, in patients with compromised immune systems or underlying chronic conditions.',
 'Monitor for complications such as thrombocytopenia and atrial fibrillation with rapid ventricular response in critically ill patients.',
 'In patients with liver disease, particularly those who have undergone liver transplantation, be aware of the risk of portal vein thrombosis and hepatorenal syndrome (HRS).',
 "Consider the potential impact of underlying conditions, such as ESRD and pulmonary hypertension, on the patient's response to treatment and overall prognosis.",
 'Be vigilant for signs of disease progression, treatment-related side effects, or paraneoplastic syndromes in patients with cancer.',
 'When evaluating patients with r

In [9]:
df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv")
print(df.shape)
print(df.columns)

(1002, 12)
Index(['File ID', 'Assessment', 'Summary', 'Subjective', 'Objective',
       'Summary_lemma', 'Summary_expanded', 'Summary_umls',
       'Summary_lemma_umls', 'Summary_expanded_umls',
       'cleaned_expanded_Summary', 'terms'],
      dtype='object')


"cleaned_expanded_Summary" (string), "terms" (List[str]) is the reference

In [3]:
df[["Assessment", "Summary", "cleaned_expanded_Summary", "terms"]][:3]

,Assessment,Summary,cleaned_expanded_Summary,terms
0,"H/O HYPERKALEMIA (HIGH POTASSIUM, HYPERPOTASSE...",# Hypoxia:; Hyperkalemia,hypoxia ; hyperkalemia,"['hypoxia', 'hyperkalemia']"
1,67 y/o M CAD s/[**Initials (NamePattern4) **] ...,Lower GI bleed; Hypotension; CAD,lower gastrointestinal bleed; hypotension; cor...,"['lower gastrointestinal bleed', 'hypotension'..."
2,81F with h/o chronic eosinophilic lung disease...,PULMONARY EMBOLISM; FEVER; HYPOTENSION; OLIGUR...,pulmonary embolism; fever; hypotension; oligur...,"['pulmonary embolism', 'fever', 'hypotension',..."


그런데 이 terms중에서도 umls내에 있는, validated한 term만을 뽑아내서 desease of interest

In [10]:
with open("/home/yl3427/cylab/SOAP_MA/data/disease_file_dict.pkl", "rb") as f:
    disease_file_dict = pickle.load(f)
top_5_diseases = sorted(
    disease_file_dict.items(), key=lambda item: len(item[1]), reverse=True
)[:5]
print("\nTop 5 diseases by number of files:")
for term, file_lst in top_5_diseases:
    print(f"{term:<50} {len(file_lst):>5}")


Top 5 diseases by number of files:
hypertension                                         132
acute renal failure                                  116
atrial fibrillation                                  100
anemia                                                90
coronary artery disease                               83


In [42]:
print(
    f"{'':<25} {'train report':>20} | {'test report (Yes)':>20} | {'test report (No)':>20}"
)
df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv")
for i in range(len(top_5_diseases)):
    term, file_lst = top_5_diseases[i]
    ltm_file_ids = file_lst[: int(len(file_lst) * 0.1)]
    test_file_ids_positive = file_lst[int(len(file_lst) * 0.1) :]
    test_file_ids_negative = list(
        set(df["File ID"].tolist()) - set(ltm_file_ids) - set(test_file_ids_positive)
    )
    print(
        f"{term:<25} {len(ltm_file_ids):>18} {len(test_file_ids_positive):>18} {len(test_file_ids_negative):>18}"
    )

                                  train report |    test report (Yes) |     test report (No)
hypertension                              13                119                870
acute renal failure                       11                105                886
atrial fibrillation                       10                 90                902
anemia                                     9                 81                912
coronary artery disease                    8                 75                919


In [7]:
terms_lst = [
    "acute renal failure",
    "anemia",
    "atrial fibrillation",
    "coronary artery disease",
    "hypertension",
]

# LTM Check

In [9]:
for term in terms_lst:
    with open(
        f"/home/yl3427/cylab/SOAP_MA/ltm_per_disease/1120_{term}_refined.pkl", "rb"
    ) as file:
        rules = pickle.load(file)
        print(f"\n{term}:")
        for rule in rules:
            print(rule)


acute renal failure:
Evaluate the patient's fluid balance by comparing total intake and output; significant imbalances, such as a negative balance indicating dehydration or a positive balance indicating fluid retention, can suggest renal dysfunction.
Monitor urine output trends; oliguria (low urine output) or anuria (no urine output) are key indicators of potential acute renal failure.
Assess serum creatinine levels and trends; an elevated or rising creatinine level is a hallmark of impaired renal function.
Consider the patient's medication history, including nephrotoxic drugs or recent changes in medication, as these can contribute to renal impairment.
Review vital signs and hemodynamic status, such as blood pressure and perfusion, as hypotension or poor perfusion can lead to acute kidney injury.
Evaluate laboratory findings, including electrolyte imbalances (e.g., hyperkalemia, acidosis) and elevated blood urea nitrogen (BUN), as these are supportive of renal dysfunction.
Consider s

In [ ]:
import pandas as pd
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv")
with open("/home/yl3427/cylab/SOAP_MA/data/disease_file_dict.pkl", "rb") as f:
    disease_file_dict = pickle.load(f)

top_5_diseases = sorted(
    disease_file_dict.items(), key=lambda item: len(item[1]), reverse=True
)[:5]

for i in range(len(top_5_diseases)):
    term, file_lst = top_5_diseases[i]
    ltm_file_ids = file_lst[: int(len(file_lst) * 0.1)]
    test_file_ids_positive = file_lst[int(len(file_lst) * 0.1) :]
    test_file_ids_negative = list(
        set(df["File ID"].tolist()) - set(ltm_file_ids) - set(test_file_ids_positive)
    )
    result_df = pd.read_csv(
        f"/home/yl3427/cylab/SOAP_MA/soap_result/1120_{term}_without_ltm.csv"
    )
    # Include 'File ID' in selection
    result_df_po = result_df[result_df["File ID"].isin(test_file_ids_positive)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]
    result_df_ne = result_df[result_df["File ID"].isin(test_file_ids_negative)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]

    # Add 'True Label' column
    result_df_po["True Label"] = "Yes"
    result_df_ne["True Label"] = "No"
    combined_df = pd.concat([result_df_po, result_df_ne], ignore_index=True)

    # Map labels to integers
    label_mapping = {"Yes": 1, "No": 0}
    true_labels = combined_df["True Label"].map(label_mapping)
    pred_labels_with_ltm = combined_df[f"{term}_with_ltm_answer"].map(label_mapping)
    pred_labels_without_ltm = combined_df[f"{term}_without_ltm_answer"].map(
        label_mapping
    )

    # Remove any rows with NaN in predicted labels
    valid_indices_with_ltm = pred_labels_with_ltm.index
    valid_indices_without_ltm = pred_labels_without_ltm.index

    # For with_ltm_answer
    precision_with_ltm = precision_score(
        true_labels.loc[valid_indices_with_ltm],
        pred_labels_with_ltm.loc[valid_indices_with_ltm],
    )
    recall_with_ltm = recall_score(
        true_labels.loc[valid_indices_with_ltm],
        pred_labels_with_ltm.loc[valid_indices_with_ltm],
    )
    f1_with_ltm = f1_score(
        true_labels.loc[valid_indices_with_ltm],
        pred_labels_with_ltm.loc[valid_indices_with_ltm],
    )

    # For without_ltm_answer
    precision_without_ltm = precision_score(
        true_labels.loc[valid_indices_without_ltm],
        pred_labels_without_ltm.loc[valid_indices_without_ltm],
    )
    recall_without_ltm = recall_score(
        true_labels.loc[valid_indices_without_ltm],
        pred_labels_without_ltm.loc[valid_indices_without_ltm],
    )
    f1_without_ltm = f1_score(
        true_labels.loc[valid_indices_without_ltm],
        pred_labels_without_ltm.loc[valid_indices_without_ltm],
    )

    # Print results
    print(f"\nMetrics for term '{term}':")
    print(f"With LTM Answer:")
    print(
        f"Precision: {precision_with_ltm:.3f}, Recall: {recall_with_ltm:.3f}, F1 Score: {f1_with_ltm:.3f}"
    )
    print(f"Without LTM Answer:")
    print(
        f"Precision: {precision_without_ltm:.3f}, Recall: {recall_without_ltm:.3f}, F1 Score: {f1_without_ltm:.3f}"
    )


Metrics for term 'hypertension':
With LTM Answer:
Precision: 0.209, Recall: 0.504, F1 Score: 0.296
Without LTM Answer:
Precision: 0.201, Recall: 0.429, F1 Score: 0.273

Metrics for term 'acute renal failure':
With LTM Answer:
Precision: 0.136, Recall: 0.876, F1 Score: 0.236
Without LTM Answer:
Precision: 0.236, Recall: 0.543, F1 Score: 0.329

Metrics for term 'atrial fibrillation':
With LTM Answer:
Precision: 0.396, Recall: 0.589, F1 Score: 0.473
Without LTM Answer:
Precision: 0.411, Recall: 0.567, F1 Score: 0.477

Metrics for term 'anemia':
With LTM Answer:
Precision: 0.085, Recall: 0.975, F1 Score: 0.156
Without LTM Answer:
Precision: 0.103, Recall: 0.926, F1 Score: 0.185

Metrics for term 'coronary artery disease':
With LTM Answer:
Precision: 0.161, Recall: 0.333, F1 Score: 0.217
Without LTM Answer:
Precision: 0.189, Recall: 0.240, F1 Score: 0.212


In [1]:
import pandas as pd
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Load the dataset
df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv")

# Load the disease file dictionary
with open("/home/yl3427/cylab/SOAP_MA/data/disease_file_dict.pkl", "rb") as f:
    disease_file_dict = pickle.load(f)

# Select the top 5 diseases based on the number of associated files
top_5_diseases = sorted(
    disease_file_dict.items(), key=lambda item: len(item[1]), reverse=True
)[:5]

# Define a label mapping
label_mapping = {"Yes": 1, "No": 0}

for term, file_lst in top_5_diseases:
    # Split the file IDs into training and testing sets
    ltm_file_ids = file_lst[: int(len(file_lst) * 0.1)]
    test_file_ids_positive = file_lst[int(len(file_lst) * 0.1) :]
    test_file_ids_negative = list(
        set(df["File ID"].tolist()) - set(ltm_file_ids) - set(test_file_ids_positive)
    )

    # Load the result dataframe for the current term
    result_df = pd.read_csv(
        f"/home/yl3427/cylab/SOAP_MA/soap_result/1120_{term}_without_ltm.csv"
    )

    # Select relevant rows and columns for positive and negative test cases
    result_df_po = result_df[result_df["File ID"].isin(test_file_ids_positive)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]
    result_df_ne = result_df[result_df["File ID"].isin(test_file_ids_negative)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]

    # Add the 'True Label' column
    result_df_po["True Label"] = "Yes"
    result_df_ne["True Label"] = "No"

    # Combine positive and negative test cases
    combined_df = pd.concat([result_df_po, result_df_ne], ignore_index=True)

    # Map textual labels to numerical labels
    combined_df["True Label"] = combined_df["True Label"].map(label_mapping)
    combined_df[f"{term}_with_ltm_answer"] = combined_df[f"{term}_with_ltm_answer"].map(
        label_mapping
    )
    combined_df[f"{term}_without_ltm_answer"] = combined_df[
        f"{term}_without_ltm_answer"
    ].map(label_mapping)

    # Drop rows with any NaN values in the predicted labels
    combined_df_with_ltm = combined_df.dropna(subset=[f"{term}_with_ltm_answer"])
    combined_df_without_ltm = combined_df.dropna(subset=[f"{term}_without_ltm_answer"])

    # Extract true labels and predictions
    true_labels = combined_df["True Label"]

    pred_with_ltm = combined_df_with_ltm[f"{term}_with_ltm_answer"]
    pred_without_ltm = combined_df_without_ltm[f"{term}_without_ltm_answer"]

    # Calculate metrics for 'with_ltm_answer'
    precision_with_ltm = precision_score(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )
    recall_with_ltm = recall_score(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )
    f1_with_ltm = f1_score(true_labels.loc[combined_df_with_ltm.index], pred_with_ltm)
    cm_with_ltm = confusion_matrix(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )

    # Calculate metrics for 'without_ltm_answer'
    precision_without_ltm = precision_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    recall_without_ltm = recall_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    f1_without_ltm = f1_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    cm_without_ltm = confusion_matrix(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )

    # Print the results
    print(f"\nMetrics for term '{term}':")

    print(f"\nWith LTM Answer:")
    print(f"Precision: {precision_with_ltm:.3f}")
    print(f"Recall: {recall_with_ltm:.3f}")
    print(f"F1 Score: {f1_with_ltm:.3f}")
    print("Confusion Matrix:")
    print(
        f"TP: {cm_with_ltm[1,1]}, FP: {cm_with_ltm[0,1]}, FN: {cm_with_ltm[1,0]}, TN: {cm_with_ltm[0,0]}"
    )

    print(f"\nWithout LTM Answer:")
    print(f"Precision: {precision_without_ltm:.3f}")
    print(f"Recall: {recall_without_ltm:.3f}")
    print(f"F1 Score: {f1_without_ltm:.3f}")
    print("Confusion Matrix:")
    print(
        f"TP: {cm_without_ltm[1,1]}, FP: {cm_without_ltm[0,1]}, FN: {cm_without_ltm[1,0]}, TN: {cm_without_ltm[0,0]}"
    )


Metrics for term 'hypertension':

With LTM Answer:
Precision: 0.209
Recall: 0.504
F1 Score: 0.296
Confusion Matrix:
TP: 60, FP: 227, FN: 59, TN: 643

Without LTM Answer:
Precision: 0.201
Recall: 0.429
F1 Score: 0.273
Confusion Matrix:
TP: 51, FP: 203, FN: 68, TN: 667

Metrics for term 'acute renal failure':

With LTM Answer:
Precision: 0.136
Recall: 0.876
F1 Score: 0.236
Confusion Matrix:
TP: 92, FP: 583, FN: 13, TN: 303

Without LTM Answer:
Precision: 0.236
Recall: 0.543
F1 Score: 0.329
Confusion Matrix:
TP: 57, FP: 185, FN: 48, TN: 701

Metrics for term 'atrial fibrillation':

With LTM Answer:
Precision: 0.396
Recall: 0.589
F1 Score: 0.473
Confusion Matrix:
TP: 53, FP: 81, FN: 37, TN: 821

Without LTM Answer:
Precision: 0.411
Recall: 0.567
F1 Score: 0.477
Confusion Matrix:
TP: 51, FP: 73, FN: 39, TN: 829

Metrics for term 'anemia':

With LTM Answer:
Precision: 0.085
Recall: 0.975
F1 Score: 0.156
Confusion Matrix:
TP: 79, FP: 851, FN: 2, TN: 61

Without LTM Answer:
Precision: 0.103
R

In [6]:
with open(
    "/home/yl3427/cylab/SOAP_MA/ltm_per_disease/1120_atrial fibrillation_refined.pkl",
    "rb",
) as file:
    ltm = pickle.load(file)
ltm

["Evaluate the patient's subjective symptoms, such as palpitations, fatigue, shortness of breath, or dizziness, as these may suggest atrial fibrillation.",
 "Assess the patient's vital signs, particularly heart rate and rhythm, for irregularities or abnormalities that may indicate atrial fibrillation.",
 "Review the patient's past medical history for conditions or events that predispose to atrial fibrillation, such as hypertension, coronary artery disease, diabetes, or previous episodes of arrhythmias.",
 "Examine the patient's medication history, including the use of anticoagulants or antiarrhythmics, as this may indicate a history of atrial fibrillation or influence its management.",
 'Perform a thorough cardiovascular examination, noting any irregular heart rhythms, murmurs, or signs of heart failure, which may be associated with atrial fibrillation.',
 'Consider laboratory findings, such as electrolyte imbalances or markers of cardiac stress, as these may contribute to or result fr

In [1]:
import pandas as pd
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Load the dataset
df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv")

# Load the disease file dictionary
with open("/home/yl3427/cylab/SOAP_MA/data/disease_file_dict.pkl", "rb") as f:
    disease_file_dict = pickle.load(f)

# Select the top 5 diseases based on the number of associated files
top_5_diseases = sorted(
    disease_file_dict.items(), key=lambda item: len(item[1]), reverse=True
)[:5]

# Define a label mapping
label_mapping = {"Yes": 1, "No": 0}

for term, file_lst in top_5_diseases:
    # Split the file IDs into training and testing sets
    ltm_file_ids = file_lst[: int(len(file_lst) * 0.1)]
    test_file_ids_positive = file_lst[int(len(file_lst) * 0.1) :]
    test_file_ids_negative = list(
        set(df["File ID"].tolist()) - set(ltm_file_ids) - set(test_file_ids_positive)
    )

    # Load the result dataframe for the current term
    result_df = pd.read_csv(
        f"/home/yl3427/cylab/SOAP_MA/soap_result/1129_{term}_without_ltm_mixtral.csv"
    )

    # Select relevant rows and columns for positive and negative test cases
    result_df_po = result_df[result_df["File ID"].isin(test_file_ids_positive)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]
    result_df_ne = result_df[result_df["File ID"].isin(test_file_ids_negative)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]

    # Add the 'True Label' column
    result_df_po["True Label"] = "Yes"
    result_df_ne["True Label"] = "No"

    # Combine positive and negative test cases
    combined_df = pd.concat([result_df_po, result_df_ne], ignore_index=True)

    # Map textual labels to numerical labels
    combined_df["True Label"] = combined_df["True Label"].map(label_mapping)
    combined_df[f"{term}_with_ltm_answer"] = combined_df[f"{term}_with_ltm_answer"].map(
        label_mapping
    )
    combined_df[f"{term}_without_ltm_answer"] = combined_df[
        f"{term}_without_ltm_answer"
    ].map(label_mapping)

    # Drop rows with any NaN values in the predicted labels
    combined_df_with_ltm = combined_df.dropna(subset=[f"{term}_with_ltm_answer"])
    combined_df_without_ltm = combined_df.dropna(subset=[f"{term}_without_ltm_answer"])

    # Extract true labels and predictions
    true_labels = combined_df["True Label"]

    pred_with_ltm = combined_df_with_ltm[f"{term}_with_ltm_answer"]
    pred_without_ltm = combined_df_without_ltm[f"{term}_without_ltm_answer"]

    # Calculate metrics for 'with_ltm_answer'
    precision_with_ltm = precision_score(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )
    recall_with_ltm = recall_score(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )
    f1_with_ltm = f1_score(true_labels.loc[combined_df_with_ltm.index], pred_with_ltm)
    cm_with_ltm = confusion_matrix(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )

    # Calculate metrics for 'without_ltm_answer'
    precision_without_ltm = precision_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    recall_without_ltm = recall_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    f1_without_ltm = f1_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    cm_without_ltm = confusion_matrix(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )

    # Print the results
    print(f"\nMetrics for term '{term}':")

    print(f"\nWith LTM Answer:")
    print(f"Precision: {precision_with_ltm:.3f}")
    print(f"Recall: {recall_with_ltm:.3f}")
    print(f"F1 Score: {f1_with_ltm:.3f}")
    print("Confusion Matrix:")
    print(
        f"TP: {cm_with_ltm[1,1]}, FP: {cm_with_ltm[0,1]}, FN: {cm_with_ltm[1,0]}, TN: {cm_with_ltm[0,0]}"
    )

    print(f"\nWithout LTM Answer:")
    print(f"Precision: {precision_without_ltm:.3f}")
    print(f"Recall: {recall_without_ltm:.3f}")
    print(f"F1 Score: {f1_without_ltm:.3f}")
    print("Confusion Matrix:")
    print(
        f"TP: {cm_without_ltm[1,1]}, FP: {cm_without_ltm[0,1]}, FN: {cm_without_ltm[1,0]}, TN: {cm_without_ltm[0,0]}"
    )


Metrics for term 'hypertension':

With LTM Answer:
Precision: 0.202
Recall: 0.445
F1 Score: 0.278
Confusion Matrix:
TP: 53, FP: 209, FN: 66, TN: 661

Without LTM Answer:
Precision: 0.160
Recall: 0.521
F1 Score: 0.245
Confusion Matrix:
TP: 62, FP: 325, FN: 57, TN: 545

Metrics for term 'acute renal failure':

With LTM Answer:
Precision: 0.148
Recall: 0.638
F1 Score: 0.240
Confusion Matrix:
TP: 67, FP: 386, FN: 38, TN: 500

Without LTM Answer:
Precision: 0.174
Recall: 0.190
F1 Score: 0.182
Confusion Matrix:
TP: 20, FP: 95, FN: 85, TN: 791

Metrics for term 'atrial fibrillation':

With LTM Answer:
Precision: 0.392
Recall: 0.567
F1 Score: 0.464
Confusion Matrix:
TP: 51, FP: 79, FN: 39, TN: 823

Without LTM Answer:
Precision: 0.392
Recall: 0.567
F1 Score: 0.464
Confusion Matrix:
TP: 51, FP: 79, FN: 39, TN: 823

Metrics for term 'anemia':

With LTM Answer:
Precision: 0.097
Recall: 0.630
F1 Score: 0.167
Confusion Matrix:
TP: 51, FP: 477, FN: 30, TN: 435

Without LTM Answer:
Precision: 0.112


In [5]:
import pandas as pd
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Load the dataset
df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv")

# Load the disease file dictionary
with open("/home/yl3427/cylab/SOAP_MA/data/disease_file_dict.pkl", "rb") as f:
    disease_file_dict = pickle.load(f)

# Select the top 5 diseases based on the number of associated files
top_5_diseases = sorted(
    disease_file_dict.items(), key=lambda item: len(item[1]), reverse=True
)[:5]

# Define a label mapping
label_mapping = {"Yes": 1, "No": 0}

for term, file_lst in top_5_diseases:
    # Split the file IDs into training and testing sets
    ltm_file_ids = file_lst[: int(len(file_lst) * 0.1)]
    test_file_ids_positive = file_lst[int(len(file_lst) * 0.1) :]
    test_file_ids_negative = list(
        set(df["File ID"].tolist()) - set(ltm_file_ids) - set(test_file_ids_positive)
    )

    # Load the result dataframe for the current term
    result_df = pd.read_csv(
        f"/home/yl3427/cylab/SOAP_MA/soap_result/1129_{term}_without_ltm_cot_mixtral.csv"
    )

    # Select relevant rows and columns for positive and negative test cases
    result_df_po = result_df[result_df["File ID"].isin(test_file_ids_positive)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]
    result_df_ne = result_df[result_df["File ID"].isin(test_file_ids_negative)][
        ["File ID", f"{term}_with_ltm_answer", f"{term}_without_ltm_answer"]
    ]

    # Add the 'True Label' column
    result_df_po["True Label"] = "Yes"
    result_df_ne["True Label"] = "No"

    # Combine positive and negative test cases
    combined_df = pd.concat([result_df_po, result_df_ne], ignore_index=True)

    # Map textual labels to numerical labels
    combined_df["True Label"] = combined_df["True Label"].map(label_mapping)
    combined_df[f"{term}_with_ltm_answer"] = combined_df[f"{term}_with_ltm_answer"].map(
        label_mapping
    )
    combined_df[f"{term}_without_ltm_answer"] = combined_df[
        f"{term}_without_ltm_answer"
    ].map(label_mapping)

    # Drop rows with any NaN values in the predicted labels
    combined_df_with_ltm = combined_df.dropna(subset=[f"{term}_with_ltm_answer"])
    combined_df_without_ltm = combined_df.dropna(subset=[f"{term}_without_ltm_answer"])

    # Extract true labels and predictions
    true_labels = combined_df["True Label"]

    pred_with_ltm = combined_df_with_ltm[f"{term}_with_ltm_answer"]
    pred_without_ltm = combined_df_without_ltm[f"{term}_without_ltm_answer"]

    # Calculate metrics for 'with_ltm_answer'
    precision_with_ltm = precision_score(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )
    recall_with_ltm = recall_score(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )
    f1_with_ltm = f1_score(true_labels.loc[combined_df_with_ltm.index], pred_with_ltm)
    cm_with_ltm = confusion_matrix(
        true_labels.loc[combined_df_with_ltm.index], pred_with_ltm
    )

    # Calculate metrics for 'without_ltm_answer'
    precision_without_ltm = precision_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    recall_without_ltm = recall_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    f1_without_ltm = f1_score(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )
    cm_without_ltm = confusion_matrix(
        true_labels.loc[combined_df_without_ltm.index], pred_without_ltm
    )

    # Print the results
    print(f"\nMetrics for term '{term}':")

    print(f"\nWith LTM Answer:")
    print(f"Precision: {precision_with_ltm:.3f}")
    print(f"Recall: {recall_with_ltm:.3f}")
    print(f"F1 Score: {f1_with_ltm:.3f}")
    print("Confusion Matrix:")
    print(
        f"TP: {cm_with_ltm[1,1]}, FP: {cm_with_ltm[0,1]}, FN: {cm_with_ltm[1,0]}, TN: {cm_with_ltm[0,0]}"
    )

    print(f"\nWithout LTM Answer:")
    print(f"Precision: {precision_without_ltm:.3f}")
    print(f"Recall: {recall_without_ltm:.3f}")
    print(f"F1 Score: {f1_without_ltm:.3f}")
    print("Confusion Matrix:")
    print(
        f"TP: {cm_without_ltm[1,1]}, FP: {cm_without_ltm[0,1]}, FN: {cm_without_ltm[1,0]}, TN: {cm_without_ltm[0,0]}"
    )


Metrics for term 'hypertension':

With LTM Answer:
Precision: 0.233
Recall: 0.118
F1 Score: 0.156
Confusion Matrix:
TP: 14, FP: 46, FN: 105, TN: 824

Without LTM Answer:
Precision: 0.258
Recall: 0.336
F1 Score: 0.292
Confusion Matrix:
TP: 40, FP: 115, FN: 79, TN: 755

Metrics for term 'acute renal failure':

With LTM Answer:
Precision: 0.147
Recall: 0.810
F1 Score: 0.249
Confusion Matrix:
TP: 85, FP: 492, FN: 20, TN: 394

Without LTM Answer:
Precision: 0.179
Recall: 0.714
F1 Score: 0.286
Confusion Matrix:
TP: 75, FP: 345, FN: 30, TN: 541

Metrics for term 'atrial fibrillation':

With LTM Answer:
Precision: 0.397
Recall: 0.600
F1 Score: 0.478
Confusion Matrix:
TP: 54, FP: 82, FN: 36, TN: 820

Without LTM Answer:
Precision: 0.398
Recall: 0.589
F1 Score: 0.475
Confusion Matrix:
TP: 53, FP: 80, FN: 37, TN: 822

Metrics for term 'anemia':

With LTM Answer:
Precision: 0.109
Recall: 0.617
F1 Score: 0.185
Confusion Matrix:
TP: 50, FP: 410, FN: 31, TN: 502

Without LTM Answer:
Precision: 0.088

In [28]:
result_df.columns

Index(['File ID', 'Assessment', 'Summary', 'Subjective', 'Objective',
       'Summary_lemma', 'Summary_expanded', 'Summary_umls',
       'Summary_lemma_umls', 'Summary_expanded_umls',
       'cleaned_expanded_Summary', 'terms', 'generated_assess',
       'hypertension_with_ltm_is_parsed', 'hypertension_with_ltm_answer',
       'hypertension_without_ltm_is_parsed',
       'hypertension_without_ltm_answer'],
      dtype='object')

In [34]:
top_5_diseases[0]

('hypertension',
 ['193604.txt',
  '189703.txt',
  '190086.txt',
  '110692.txt',
  '187277.txt',
  '105738.txt',
  '147696.txt',
  '186757.txt',
  '197276.txt',
  '185114.txt',
  '190524.txt',
  '105852.txt',
  '148588.txt',
  '184814.txt',
  '102324.txt',
  '109844.txt',
  '111039.txt',
  '149619.txt',
  '111314.txt',
  '111857.txt',
  '148314.txt',
  '149546.txt',
  '110341.txt',
  '147435.txt',
  '148470.txt',
  '198734.txt',
  '184453.txt',
  '111845.txt',
  '198330.txt',
  '103677.txt',
  '192754.txt',
  '105172.txt',
  '194131.txt',
  '103853.txt',
  '101063.txt',
  '100590.txt',
  '101324.txt',
  '183333.txt',
  '194061.txt',
  '106110.txt',
  '104263.txt',
  '184503.txt',
  '148647.txt',
  '111040.txt',
  '101737.txt',
  '186671.txt',
  '194088.txt',
  '185556.txt',
  '106883.txt',
  '191719.txt',
  '199580.txt',
  '185625.txt',
  '189180.txt',
  '194847.txt',
  '195768.txt',
  '105347.txt',
  '148251.txt',
  '199781.txt',
  '197960.txt',
  '149006.txt',
  '183813.txt',
  '1130

In [35]:
"190090.txt" in top_5_diseases[0][1]

False

In [11]:
result_df = pd.read_csv(
    f"/home/yl3427/cylab/SOAP_MA/soap_result/1206_hypertension_so_a_mixtral.csv"
)

for _, row in result_df.iterrows():
    if pd.isnull(row["terms"]) or "hypertension" not in row["terms"]:
        continue
    print(f"File ID: {row['File ID']}")
    print("=" * 80)
    print(f"Subjective:\n{row['Subjective']}\n")
    print(f"Objective:\n{row['Objective']}\n")
    print(f"Assessment:\n{row['Assessment']}\n")
    print(f"Reference Summary:\n{row['terms']}\n")
    print(
        f"Is Hypertension? -> Reference Answer: {'Yes' if row['File ID'] in top_5_diseases[0][1] else 'No'}\n"
    )
    print(f"Generated Assessment:\n{row['generated_assess']}\n")
    print(f"Generated Answer (with LTM): {row['hypertension_with_ltm_answer']}\n")
    print(f"Generated Answer (without LTM): {row['hypertension_without_ltm_answer']}\n")
    print(f"Generated Summary:\n{row['generated_summary']}\n")
    print("-----THE END-----")
    print()

File ID: 190090.txt
Subjective:
4 Hour Events:
- Placed on Heparin with goal of maintaining PTT of 60-80; changed from
weight based to heparin sliding scale.
- LENI:  DVT in left lower extremity.
- Baseline CXR.
-          Follow up ECHO
- Weaned O2 from NRB
3LNC with no desats
-Reported having back pain (chronic issue related to bulging lumbar
disks, no radicular symptoms); left shoulder pain (tpa in L arm) with
mildly tender, induration over left anterior deltoid; pleuritic CP
(incr with deep breathing, coughing) 0/10 at b/l with incr to [**2196-6-6**];
frontal headache (started around 2am
.[**2196-6-6**] initially
[**2-9**] now). Mild
nausea O/N. Mild non-productive cough.
Denies SOB, lightheadedness, dizziness, abdominal pain.
Changes to  and
f
Review of systems is unchanged from admission except as noted below
Review of systems:

Objective:
Last dose of Antibiotics:
Infusions:
Heparin Sodium - 1,500 units/hour
Other ICU medications:
Other medications:
Flowsheet Data as of  [**2194

In [4]:
df["Summary"]

0                                # Hypoxia:; Hyperkalemia
1                        Lower GI bleed; Hypotension; CAD
2       PULMONARY EMBOLISM; FEVER; HYPOTENSION; OLIGUR...
3       Sepsis; Altered/Depressed MS\n thought to be [...
4       # Bradycardia / Rhythm; #. Hypertension; # CAD...
                              ...                        
997     Resp distress; recurrent asp pna/pneumonitis; ...
998     Hypoxia; Fever: Most likely infectious with ei...
999     Critical Aortic Stenosis; Transaminitis; Hypon...
1000    Respiratory failure; mucous plugging\n   and G...
1001    Hyponatremia: Likely acute on chronic hyponatr...
Name: Summary, Length: 1002, dtype: object

# check the number of token in ltm (openai models tokenizer)

In [18]:
import tiktoken

enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")
len(enc.encode("how?"))

2

In [3]:
with open(
    "/home/yl3427/cylab/SOAP_MA/ltm_per_disease/acute_renal_failure_raw.pkl", "rb"
) as f:
    arf_ltm_raw = pickle.load(f)

In [20]:
import numpy as np
from scipy import stats

lengths = [len(enc.encode(rule)) for rule in arf_ltm_raw]
min_len = np.min(lengths)
max_len = np.max(lengths)
mean_len = np.mean(lengths)
mode_len = stats.mode(lengths)

print(f"Min length: {min_len}")
print(f"Max length: {max_len}")
print(f"Mean length: {mean_len}")
print(f"Mode length: {mode_len}")

Min length: 15
Max length: 37
Mean length: 24.442857142857143
Mode length: ModeResult(mode=22, count=15)


In [9]:
with open(
    "/home/yl3427/cylab/SOAP_MA/ltm_per_disease/acute_renal_failure_refined.pkl", "rb"
) as f:
    arf_ltm_refined = pickle.load(f)

len(arf_ltm_refined)

15

In [21]:
lengths = [len(enc.encode(rule)) for rule in arf_ltm_refined]
min_len = np.min(lengths)
max_len = np.max(lengths)
mean_len = np.mean(lengths)
mode_len = stats.mode(lengths)

print(f"Min length: {min_len}")
print(f"Max length: {max_len}")
print(f"Mean length: {mean_len}")
print(f"Mode length: {mode_len}")

Min length: 14
Max length: 32
Mean length: 23.466666666666665
Mode length: ModeResult(mode=22, count=2)


In [ ]:
# 하나의 룰당 최대 50token이면 되겠네